In [5]:
#designed to have less randomness
from gefs.pc import PC
import pandas as pd
import numpy as np
from experiments.prep import get_stats, normalize_data, standardize_data, learncats, get_dummies
import time
from gefs.trees import RandomForest
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from experiments.prep import get_data
from sklearn.model_selection import KFold

C:\Users\Mennolt van alten\Documents\Data Science - Universiteit\Year 3\BEP\GeFs_Repo\gefs\cluster.py:91: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  ('thr', optional(float64)),
C:\Users\Mennolt van alten\Documents\Data Science - Universiteit\Year 3\BEP\GeFs_Repo\gefs\signed.py:11: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  ('size', int64)
C:\Users\Mennolt van alten\Documents\Data Science - Unive

In [18]:
#first divide data into 10 folds, make 10 random seeds for np rng
def full_test(depths, nr_trees, file_name, file_path = "data/", seed = 1, save_path = "", **kwargs):
    """
    Does a full test, for combinations of depth and nr of trees, for file named 'name'
    depths = an iterable containing all depths to be looked into
    nr_trees = an iterable containing all numbers of trees to be looked into
    file_name = the name of the data file
    file_path = the path to the data file
    seed = the seed to seed all rng with
    save_path = the path in which to save results
    """
    start = time.time()
    #get the data & make folds
    data, ncat = get_data(file_name, "data/")
    splitter = KFold(10, random_state = seed, shuffle = True)
    
    for depth in depths:
        print(depth, "start")
        #loop over the number of trees
        performance, accuracy = hyperparameter_range_test(data, ncat, nr_trees, splitter, depth, seed,  **kwargs,
                                                          min_samples_leaf = depth) 
        
        #create csv files about performance in both likelihood and accuracy
        lls = ["ll"+str(i) for i in range(10)]
        performance_df = pd.DataFrame(performance, index = nr_trees, columns = lls)
        performance_df['llmean'] = [sum(i)/len(i) for i in performance]
        performance_df['llvariance'] = performance_df.apply(lambda x : variance(lls, 'llmean', x), axis = 1)
        performance_df['values'] = nr_trees
        
        acs = ["ac"+str(i) for i in range(10)]
        acc_df = pd.DataFrame(accuracy, index = nr_trees, columns = acs)
        acc_df['acmean'] = [sum(i)/len(i) for i in accuracy]
        acc_df['acvariance'] = acc_df.apply(lambda x : variance(acs, 'acmean', x), axis = 1)
        acc_df['values'] = nr_trees

        #create scatter plot of the data
#         tested = "number of trees"
#         fig, ax = plt.subplots(ncols = 2, figsize = (15, 7))
#         performance_df.plot(kind = 'scatter', x = "values", y = 0, ax = ax[0])
#         performance_df.plot(kind = 'scatter', x = "values", y = 1, ax = ax[0])
#         performance_df.plot(kind = 'scatter', x = "values", y = 2, ax = ax[0])
#         performance_df.plot(kind = 'scatter', x = "values", y = "llmean", color = "orange", title = f"{file_name}, one train set, depth {depth}", xlabel = tested, ax = ax[0])
#         ax[0].legend(["0", "1", "2", "mean"])
#         acc_df.plot(kind = 'scatter', x = "values", y = 0, ax = ax[1])
#         acc_df.plot(kind = 'scatter', x = "values", y = 1, ax = ax[1])
#         acc_df.plot(kind = 'scatter', x = "values", y = 2, ax = ax[1])
#         acc_df.plot(kind = 'scatter', x = "values", y = "acmean", color = "orange", title = f"{file_name}, one train set, depth {depth}", xlabel = tested, ax = ax[1])
#         ax[1].legend(["0", "1", "2", "mean"])
#         #ax.set_xlim(0,200)
#         #ax.set_ylim(-60.5, -62.5)
#         cur_moment = str(datetime.datetime.today())[0:19].replace(":", "_")
#         name = f"{cur_moment}_depth_{depth}_{file_name[0:-4]}"
#         plt.savefig(f"../graphs/{name}.png")
#         plt.close()
        cur_moment = str(datetime.datetime.today())[0:19].replace(":", "_")
        name = f"{cur_moment}_depth_{depth}_{file_name[0:-4]}"
        new_df = acc_df.join(performance_df[performance_df.columns[:-1]])
        new_df.to_csv(f"{save_path}{name}.csv")
        
        end_string = f"\n\ndepth {depth} end: {int(time.time()-start)} seconds \n\n\n"
        print(end_string)
    
    #for train_index, test_index in splitter.split(data):

def hyperparameter_range_test(data, n_cat, values, splitter, depth = None, seed = 1, 
                              save_train_performance = False, mode = "depth", min_samples_leaf = None, **kwargs):
    """
    input:
    Data: 
        some dataset to make a model from
    n_cat: 
        number of categories for each column in the data 
        (get using learncats, for more see experiments.prep.learncats and experiments.prep.get_data)
    Values: hyperparameter values to try (right now trying them on n_estimators)
    depth: what the depth of the tries should be
    splitter: a KFold object used to split the data into folds
    save_train_performance: whether to save data on the performance on the training data rather than on the testing data
    mode: what parameter to use to limit tree height. supported: depth, min_samples_per_leaf
    seed: random seed to be used at the start of each test"""
    
    start = time.time()
#     assert train_ratio >= 0
#     assert train_ratio <= 1
#     shuffle = np.random.choice(range(data.shape[0]), data.shape[0], replace=False)
#     data_train = data[shuffle[:int(train_ratio*data.shape[0])], :]
#     #splitting test data into dependent and independent variables
#     X_train, y_train = data_train[:, :-1], data_train[:, -1] 
#     data_test = data[shuffle[int(train_ratio*data.shape[0]):], :]
#     #making test data not have a class:
#     correct_class_test = data_test[:, -1].copy()
#     data_test[:, -1] = np.nan
#     print("data split")
    print(kwargs, mode)
    performance = []
    accuracy = []
    for i in values:
        #make a new temporary array for saving
        temp_performance = []
        temp_accuracy = [] #accuracy on the original classification
        j = 0
        for train_index, test_index in splitter.split(data):
            j = j + 1
            #redistribute the data into the folds
            data_train = data[train_index]
            X_train, y_train = data_train[:, :-1], data_train[:, -1] 
            if save_train_performance == False:
                data_test = data[test_index]
                correct_class_test = data_test[:, -1].copy()
                data_test[:, -1] = np.nan
            else:
                data_test = data[train_index]
                correct_class_test = data_test[:,-1].copy()
                data_test[:,-1]=np.nan
                print("Warning, data will be saved for training set instead of test set")
            
            

#             shuffle = np.random.choice(range(data.shape[0]), data.shape[0], replace=False)
#             data_train = data[shuffle[:int(train_ratio*data.shape[0])], :]
#             #splitting test data into dependent and independent variables
#             X_train, y_train = data_train[:, :-1], data_train[:, -1] 
#             data_test = data[shuffle[int(train_ratio*data.shape[0]):], :]
#             #making test data not have a class:
#             data_test[:, -1] = np.nan
#             print(f"model {i}, {j} data distributed")
            #set a seed
            np.random.seed(seed)
            #make a model (pick what to change based on conditions)
            if mode == "depth":
                model = RandomForest(n_estimators=i, ncat=n_cat, max_depth = depth)
                print("Running in depth mode")
            elif mode == "min_samples_leaf":
                model = RandomForest(n_estimators=i, ncat=n_cat, min_samples_leaf = min_samples_leaf)
                print("Running in min_samples_per_leaf mode")
            else:
                raise ValueError("No mode given")
            model.fit(X_train, y_train)
            print(f"model {i}, {j} fitted")
            print(int(time.time()-start), "seconds")
            #turn it to a probability circuit
            circuit = model.topc(learnspn=np.Inf) #note: here we can set max_height (default 1000000)
            print(f"model {i}, {j} circuited")
            print(int(time.time()-start), "seconds")
            #test the model
            #return(circuit, data_test, correct_class_test)
            temp_performance.append(circuit.log_likelihood(data_test).mean())
            predictions = circuit.classify(data_test[:, :-1])
            residuals = [x == y for (x,y) in zip(predictions, correct_class_test)]#abs(predictions-correct_class_test)
            cur_accuracy = sum(residuals)/len(residuals)#1-(sum(residuals)/len(residuals))
            temp_accuracy.append(cur_accuracy)
            print(f"model {i}, {j} tested: acc {cur_accuracy}, log-lik {temp_performance[-1]}")
            print(int(time.time()-start), "seconds")
            #delete model to try again
            circuit.delete()
            model.delete()
            del(model)
        #and save the test results into performance
        performance.append(temp_performance)
        accuracy.append(temp_accuracy)
       
        print(f"{i} finished, {int(time.time()-start)} seconds")
        print(f"Performance: {performance[-1]}")
        
    return(performance, accuracy)

def variance(cols, mean_col, row):
    var = 0
    for col in cols:
        var = var + (row[col]-row[mean_col])**2
    return(var)

In [21]:
depths = [1,5,10,30]#[i for i in range(1,11)]
nr_trees = [10, 25, 50, 75, 100]#[10,50,100]
name = "diabetes.csv"
full_test(depths, nr_trees, name, save_path = f"Experiment2_min_samples/{name[:-4]}_training/", mode = "min_samples_leaf", save_train_performance = True)

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 132.74it/s]

1 start
{} min_samples_leaf
Warning, data will be saved for training set instead of test set
Running in min_samples_per_leaf mode
model 10, 1 fitted
0 seconds
model 10, 1 circuited
0 seconds



100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 140.60it/s]

model 10, 1 tested: acc 0.9942112879884226, log-lik -20.195553068459763
0 seconds
Warning, data will be saved for training set instead of test set
Running in min_samples_per_leaf mode
model 10, 2 fitted
0 seconds
model 10, 2 circuited
0 seconds



  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

...

model 100, 9 fitted
19 seconds
model 100, 9 circuited
19 seconds


model 100, 9 tested: acc 0.7991329479768786, log-lik -26.991343808171905
20 seconds
Warning, data will be saved for training set instead of test set
Running in min_samples_per_leaf mode


100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 734.51it/s]

model 100, 10 fitted
20 seconds
model 100, 10 circuited
20 seconds


model 100, 10 tested: acc 0.796242774566474, log-lik -26.766824661121362
20 seconds
100 finished, 21 seconds
Performance: [-26.789339306837075, -26.89124460163738, -26.952988234241037, -27.003979027187743, -26.7872601048747, -26.861983390640372, -26.811387973380864, -26.962492790254664, -26.991343808171905, -26.766824661121362]


depth 30 end: 193 seconds 



